# Backpropagation

In [ ]:
import numpy as np
import pandas as pd
import time

Pada post-test kali ini akan membandingkan dua jenis fungsi aktivasi yang biasa digunakan dalam backpropogation

In [76]:
#Fungsi Aktivasi Sigmoid dengan turunannya
def sig(x):
    return 1 / (1 + np.exp(-x))

def sigd(x):
    return sig(x) * (1 - sig(x))
#Fungsi Aktivasi Hyperbolic Tangent dengan turunannya

def ht(x):
    return np.tanh(x)

def htd(x):
    return 1 - np.tanh(x) ** 2

In [77]:
def onehot_enc(lbl, min_val=0):
  mi = min(lbl)
  enc = np.full((len(lbl), max(lbl) - mi + 1), min_val, np.int8)

  for i, x in enumerate(lbl):
    enc[i, x - mi] = 1

  return enc

def onehot_dec(enc, mi=0):
  return [np.argmax(e) + mi for e in enc]

### a) Fungsi *Training* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [78]:
def bp_fit_sig(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, print_per_epoch=100):
    start_time = time.time()
    np.random.seed(1)

    nin = [np.empty(i) for i in layer_conf]
    n = [np.empty(j + 1) if i < len(layer_conf) - 1 else np.empty(j) for i, j in enumerate(layer_conf)]
    w = [np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)]
    dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)]
    d = [np.empty(s) for s in layer_conf[1:]]
    din = [np.empty(s) for s in layer_conf[1:-1]]
    epoch = 0
    mse = 1

    for i in range(0, len(n)-1):
        n[i][-1] = 1

    while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
        epoch += 1
        mse = 0

        for r in range(len(X)):
            n[0][:-1] = X[r]

            for L in range(1, len(layer_conf)):
                nin[L] = np.dot(n[L-1], w[L-1])
                n[L][:len(nin[L])] = sig(nin[L])
            e = target[r] - n[-1]
            mse += sum(e ** 2)
            d[-1] = e * sigd(nin[-1])
            dw[-1] = learn_rate * np.outer(n[-2], d[-1])

            for L in range(len(layer_conf) - 2, 0, -1):
                din[L-1] = np.dot(d[L], w[L][:-1].T)
                d[L-1] = din[L-1] * sigd(nin[L])
                dw[L-1] = learn_rate * np.outer(n[L-1], d[L-1])

            for i in range(len(w)):
                w[i] += dw[i]

        mse /= len(X)
        if print_per_epoch > -1 and epoch % print_per_epoch == 0:
            print(f'Epoch {epoch}, MSE: {mse}')

    execution = time.time() - start_time
    print("Waktu eksekusi: %s detik" % execution)
    return w, epoch, mse

In [79]:
#Membuat fungsi training backpropagation dengan menggunakan fungsi aktivasi tanh
def bp_fit_tanh(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, print_per_epoch=100):
    start_time = time.time()
    np.random.seed(1)
    nin = [np.empty(i) for i in layer_conf]
    n = [np.empty(j + 1) if i < len(layer_conf) - 1 else np.empty(j) for i, j in enumerate(layer_conf)]
    w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)])
    dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)]
    d = [np.empty(s) for s in layer_conf[1:]]
    din = [np.empty(s) for s in layer_conf[1:-1]]
    epoch = 0
    mse = 1
    for i in range(0, len(n)-1):
        n[i][-1] = 1
    while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
        epoch += 1
        mse = 0
        for r in range(len(X)):
            n[0][:-1] = X[r]
            for L in range(1, len(layer_conf)):
                nin[L] = np.dot(n[L-1], w[L-1])
                n[L][:len(nin[L])] = ht(nin[L])
            e = target[r] - n[-1]
            mse += sum(e ** 2)
            d[-1] = e * htd(nin[-1])
            dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))
            for L in range(len(layer_conf) - 1, 1, -1):
                din[L-2] = np.dot(d[L-1], np.transpose(w[L-1][:-1]))
                d[L-2] = din[L-2] * np.array(htd(nin[L-1]))
                dw[L-2] = (learn_rate * d[L-2]) * n[L-2].reshape((-1, 1))
            w += dw
        mse /= len(X)
        if print_per_epoch > -1 and epoch % print_per_epoch == 0:
            print(f'Epoch {epoch}, MSE: {mse}')
    execution = time.time() - start_time
    print("Waktu eksekusi: %s detik" % execution)
    return w, epoch, mse

### b) Fungsi *Testing* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [80]:
def bp_predict_sig(X, w):
  n = [np.empty(len(i)) for i in w]
  nin = [np.empty(len(i[0])) for i in w]
  predict = []
  n.append(np.empty(len(w[-1][0])))
  for x in X:
    n[0][:-1] = x
    for L in range(0, len(w)):
      nin[L] = np.dot(n[L], w[L])
      n[L + 1][:len(nin[L])] = sig(nin[L])
    predict.append(n[-1].copy())
  return predict

In [81]:
#Membuat fungsi testing backpropagation dengan menggunakan fungsi aktivasi tanh
def bp_predict_tanh(X, w):
  n = [np.empty(len(i)) for i in w]
  nin = [np.empty(len(i[0])) for i in w]
  predict = []
  n.append(np.empty(len(w[-1][0])))
  for x in X:
    n[0][:-1] = x
    for L in range(0, len(w)):
      nin[L] = np.dot(n[L], w[L])
      n[L + 1][:len(nin[L])] = ht(nin[L])
    predict.append(n[-1].copy())
  return predict

### c) Klasifikasi dataset wine


Lakukan pelatihan pada dataset wine dengan menggunakan 2 fungsi pelatihan yang telah dibuat!

Konfigurasi kedua pelatihan harus sama (epoch, hidden layer, learning rate, dll).
Akurasi yang diharapkan di setiap pelatihan adalah > 0.98

In [82]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

wine = datasets.load_wine()
X = minmax_scale(wine.data)
Y = onehot_enc(wine.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y,
test_size=.3,random_state=1)
#Isi jumlah layer yang digunakan dengan jumlah hidden layer #
w, ep, mse = bp_fit_sig(X_train, y_train,
                        layer_conf=(13, 6, 3),
                        learn_rate=0.01,
                        max_epoch=1000,
                        max_error=0.001,
                        print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict_sig(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Epoch 25, MSE: 0.6579941238294782
Epoch 50, MSE: 0.6572768161195099
Epoch 75, MSE: 0.6565545661287119
Epoch 100, MSE: 0.6557770192506821
Epoch 125, MSE: 0.654899417791867
Epoch 150, MSE: 0.6538595384431759
Epoch 175, MSE: 0.6525614618713684
Epoch 200, MSE: 0.6508427367655694
Epoch 225, MSE: 0.6484000824227005
Epoch 250, MSE: 0.64460019717423
Epoch 275, MSE: 0.6379349086861484
Epoch 300, MSE: 0.6243884850349829
Epoch 325, MSE: 0.5952654595950061
Epoch 350, MSE: 0.5486251657404897
Epoch 375, MSE: 0.49598873444689995
Epoch 400, MSE: 0.44628364360017614
Epoch 425, MSE: 0.40239290598001787
Epoch 450, MSE: 0.3626842811835349
Epoch 475, MSE: 0.32539923340438925
Epoch 500, MSE: 0.290301127018108
Epoch 525, MSE: 0.25814309340278824
Epoch 550, MSE: 0.2297068402933371
Epoch 575, MSE: 0.2052706419752029
Epoch 600, MSE: 0.18460691798570156
Epoch 625, MSE: 0.1672162399817934
Epoch 650, MSE: 0.1525380379526155
Epoch 675, MSE: 0.1400621131490867
Epoch 700, MSE: 0.12936519208153907
Epoch 725, MSE: 0.12

In [83]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

wine = datasets.load_wine()
X = minmax_scale(wine.data)
Y = onehot_enc(wine.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y,
test_size=.3,random_state=1)
#Isi jumlah layer yang digunakan dengan jumlah hidden layer #
w, ep, mse = bp_fit_sig(X_train, y_train,
                        layer_conf=(13, 6, 3),
                        learn_rate=0.01,
                        max_epoch=1000,
                        max_error=0.001,
                        print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict_tanh(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Epoch 25, MSE: 0.6579941238294782
Epoch 50, MSE: 0.6572768161195099
Epoch 75, MSE: 0.6565545661287119
Epoch 100, MSE: 0.6557770192506821
Epoch 125, MSE: 0.654899417791867
Epoch 150, MSE: 0.6538595384431759
Epoch 175, MSE: 0.6525614618713684
Epoch 200, MSE: 0.6508427367655694
Epoch 225, MSE: 0.6484000824227005
Epoch 250, MSE: 0.64460019717423
Epoch 275, MSE: 0.6379349086861484
Epoch 300, MSE: 0.6243884850349829
Epoch 325, MSE: 0.5952654595950061
Epoch 350, MSE: 0.5486251657404897
Epoch 375, MSE: 0.49598873444689995
Epoch 400, MSE: 0.44628364360017614
Epoch 425, MSE: 0.40239290598001787
Epoch 450, MSE: 0.3626842811835349
Epoch 475, MSE: 0.32539923340438925
Epoch 500, MSE: 0.290301127018108
Epoch 525, MSE: 0.25814309340278824
Epoch 550, MSE: 0.2297068402933371
Epoch 575, MSE: 0.2052706419752029
Epoch 600, MSE: 0.18460691798570156
Epoch 625, MSE: 0.1672162399817934
Epoch 650, MSE: 0.1525380379526155
Epoch 675, MSE: 0.1400621131490867
Epoch 700, MSE: 0.12936519208153907
Epoch 725, MSE: 0.12

# Pertanyaan

1.  Apa perbedaan dari penggunaan fungsi aktivasi sigmoid dengan fungsi aktivasi hyperbolic tangent?
2. Coba jelaskan alasan dari perbedaan tersebut sebisa kalian

# Jawaban

1.  Fungsi aktivasi sigmoid menghasilkan nilai antara 0 dan 1, membentuk kurva yang menyerupai huruf S. Ketika nilai input semakin besar, outputnya akan semakin mendekati 1. Sebaliknya, jika input semakin kecil, outputnya akan mendekati 0. Fungsi tanh juga menghasilkan kurva berbentuk S, namun lebih simetris. Rentang outputnya antara -1 dan 1, sehingga dapat merepresentasikan nilai positif maupun negatif.

2. Perbedaan utama antara sigmoid dan tanh terletak pada rentang nilai outputnya. Sigmoid menghasilkan nilai antara 0 dan 1, membuatnya ideal untuk tugas klasifikasi biner karena outputnya dapat diinterpretasikan sebagai probabilitas. Sebaliknya, tanh menghasilkan nilai antara -1 dan 1, yang memungkinkan representasi data yang lebih kompleks dan simetris. Simetri ini serta gradien yang lebih besar pada tanh membuatnya lebih efektif dalam mengatasi masalah vanishing gradient yang sering terjadi pada jaringan saraf dalam. Oleh karena itu, sigmoid umumnya digunakan pada lapisan output, sementara tanh lebih sering digunakan pada lapisan tersembunyi.

Pilihan antara sigmoid dan tanh tergantung pada jenis masalah yang ingin kita selesaikan. Jika kita memiliki masalah klasifikasi biner, sigmoid adalah pilihan yang baik. Namun, untuk masalah yang lebih kompleks, seperti pengenalan gambar atau pemrosesan bahasa alami, tanh atau fungsi aktivasi lainnya yang memiliki gradien lebih besar, seperti ReLU, biasanya lebih disukai.